# Collect all settled locations

In [202]:
import geopandas as gpd
import pandas as pd
import geopy
import osmnx as ox
import numpy as np
import rasterio as rio
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp').sample(100)
location_csv = gpd.GeoDataFrame()
location_csv['name'] = iq_locations['PNameEn']
location_csv['region'] = iq_locations['A1NameEn']
location_csv['country'] = 'Iraq'
location_csv['lat'] = iq_locations['Latitude']
location_csv['lon'] = iq_locations['Longitude']
location_csv['location_type'] = 'town'
location_csv['conflict_date'] = 0
location_csv['pop/cap'] = 0
location_csv.head()

,name,region,country,lat,lon,location_type,conflict_date,pop/cap
18147,Al Halfayah,Missan,Iraq,31.82,47.43,town,0,0
21307,Jubur Sharrad,Wassit,Iraq,33.03,45.91,town,0,0
14584,Sharayi,Erbil,Iraq,36.08,43.58,town,0,0
6066,Kfradoli Lower,Sulaymaniyah,Iraq,36.22,44.76,town,0,0
4323,Shawk,Sulaymaniyah,Iraq,35.21,44.91,town,0,0


# Get population numbers

In [203]:
pop87 = pd.read_csv('data/undata_1987_census/UNdata_Export_20180505_224541956.csv')
pop87 = pop87[:-2] # drop nans

In [204]:
'''replace_dict'''

'replace_dict'

In [205]:
'''iq_pop_bydist = gpd.read_file('data/Iraq_Districts/iraq_districts.shp')
iq_pop_bydist['num_settlements'] = 0
iq_pop_bydist['pop87'] = 0
iq_pop_bydist['pop_2015'] = 0
iq_pop_bydist['pred_pop'] = 0

replace_dict = {}
# this had to be done by hand to check spellings and varied Eng-Arabic transliteration
replace_dict['Adhamiya'] = pop87.Value[pop87.City== 'Adhamiyah']
replace_dict['Amara'] = pop87.Value[pop87.City== 'Amara']
# districts not given unique counts in 87: 
replace_dict['Tarmia'] = pop87.Value[pop87.City== 'Baghdad'] / 
replace_dict['Khadamiya'] = pop87.Value[pop87.City== 'Kadhimain']
replace_dict[''] = pop87.Value[pop87.City== 'Karradah Sharqiyah']'''

"iq_pop_bydist = gpd.read_file('data/Iraq_Districts/iraq_districts.shp')\niq_pop_bydist['num_settlements'] = 0\niq_pop_bydist['pop87'] = 0\niq_pop_bydist['pop_2015'] = 0\niq_pop_bydist['pred_pop'] = 0\n\nreplace_dict = {}\n# this had to be done by hand to check spellings and varied Eng-Arabic transliteration\nreplace_dict['Adhamiya'] = pop87.Value[pop87.City== 'Adhamiyah']\nreplace_dict['Amara'] = pop87.Value[pop87.City== 'Amara']\n# districts not given unique counts in 87: \nreplace_dict['Tarmia'] = pop87.Value[pop87.City== 'Baghdad'] / \nreplace_dict['Khadamiya'] = pop87.Value[pop87.City== 'Kadhimain']\nreplace_dict[''] = pop87.Value[pop87.City== 'Karradah Sharqiyah']"

In [206]:
x = np.array([[1, 2, 3, 4], [4,4,4,4]])
x.shape

(2, 4)

# Above was taking too long, so just going to use worldpop estimates,  crs matches for locations and worldpop raster

In [207]:
# 2015 projections from WorldPop, adjusted to UN estimates
pops_raster = rio.open('data/IRQ_ppp_v2b_2015_UNadj/IRQ_ppp_v2b_2015_UNadj.tif')


In [208]:
'''#http://www.perrygeo.com/python-affine-transforms.html
pop_pred = []
pops_dat = pops.read().squeeze()
translator = pops.affines
with open('pops_pred.txt', 'w') as f:
    for y in range(pops_dat.shape[0]):
        for x in range(pops_dat.shape[1]):
            long, lat = translator * (x,y) # remember to swap b/c this is x,y
            f.write(",".join([str(long), str(lat), str(pops_dat[y,x])]))
# takes too long, try another way'''

'#http://www.perrygeo.com/python-affine-transforms.html\npop_pred = []\npops_dat = pops.read().squeeze()\ntranslator = pops.affines\nwith open(\'pops_pred.txt\', \'w\') as f:\n    for y in range(pops_dat.shape[0]):\n        for x in range(pops_dat.shape[1]):\n            long, lat = translator * (x,y) # remember to swap b/c this is x,y\n            f.write(",".join([str(long), str(lat), str(pops_dat[y,x])]))\n# takes too long, try another way'

In [209]:
translator = pops_raster.affine
pops_dat = pops_raster.read().squeeze()
iq_locs_pop = location_csv.copy()
pops = []
for i in iq_locs_pop.itertuples():
    x = i.lon
    y = i.lat
    col, row = ~ translator * (x, y)
    col = int(round(col,0))
    row = int(round(row,0))
    pops.append(pops_dat[row, col])
    
location_csv['pop_density'] = pops
location_csv.to_csv('data/iq_locs_pop.csv')

In [210]:
location_csv.head()

,name,region,country,lat,lon,location_type,conflict_date,pop/cap,pop_density
18147,Al Halfayah,Missan,Iraq,31.82,47.43,town,0,0,2.522590
21307,Jubur Sharrad,Wassit,Iraq,33.03,45.91,town,0,0,0.101088
14584,Sharayi,Erbil,Iraq,36.08,43.58,town,0,0,3.333604
6066,Kfradoli Lower,Sulaymaniyah,Iraq,36.22,44.76,town,0,0,2.075362
4323,Shawk,Sulaymaniyah,Iraq,35.21,44.91,town,0,0,0.358639


# now getting the routes between all locations
start with just doing great circle distance, then maybe move to routing values

In [211]:
dist_iq_locations = iq_locations.copy()

In [212]:
'''dist_df = pd.DataFrame()
for i in dist_iq_locations.itertuples():
    dist_df[i.PNameEn] = dist_iq_locations.geometry.distance(i.geometry)
dist_df.index = dist_df.columns
dist_df.to_csv('distance_matrix.csv')'''

"dist_df = pd.DataFrame()\nfor i in dist_iq_locations.itertuples():\n    dist_df[i.PNameEn] = dist_iq_locations.geometry.distance(i.geometry)\ndist_df.index = dist_df.columns\ndist_df.to_csv('distance_matrix.csv')"

In [213]:
trips = set()
count = 0
with open("flee-release/routes.csv", "w") as f:
    #f.write(",".join(["name1", "name2", "distance\n"]))
    for i in dist_iq_locations.itertuples():
        dists = dist_iq_locations.geometry.distance(i.geometry) * 1000
        dists = [int(round(i,0)) for i in dists]
        tups = zip(dist_iq_locations.PNameEn, dists)
        for destination, time in tups:
            key = (i.PNameEn, destination)
            if key[0] == key[1]:
                continue
            if key not in trips:
                trips.add(key)
                trips.add((key[1], key[0]))
                f.write(",".join([key[0], key[1], str(time)]) + "\n")
                if key[0] != key[1]:
                    f.write(",".join([key[1], key[0], str(time)]) + "\n")

# conflict locations for location_csv

In [214]:
'''conflict_loc = pd.read_csv('https://data.humdata.org/hxlproxy/data.csv?url=https%3A%2F%2Fapi.acleddata.com%2Facled%2Fread.csv%3Flimit%3D0%26iso%3D368&name=ACLEDHXL&tagger-match-all=on&tagger-02-header=iso&tagger-02-tag=%23country%2Bcode&tagger-03-header=event_id_cnty&tagger-03-tag=%23event%2Bcode&tagger-05-header=event_date&tagger-05-tag=%23date%2Boccurred+&tagger-08-header=event_type&tagger-08-tag=%23event%2Btype&tagger-09-header=actor1&tagger-09-tag=%23group%2Bname%2Bfirst&tagger-10-header=assoc_actor_1&tagger-10-tag=%23group%2Bname%2Bfirst%2Bassoc&tagger-12-header=actor2&tagger-12-tag=%23group%2Bname%2Bsecond&tagger-13-header=assoc_actor_2&tagger-13-tag=%23group%2Bname%2Bsecond%2Bassoc&tagger-16-header=region&tagger-16-tag=%23region%2Bname&tagger-17-header=country&tagger-17-tag=%23country%2Bname&tagger-18-header=admin1&tagger-18-tag=%23adm1%2Bname&tagger-19-header=admin2&tagger-19-tag=%23adm2%2Bname&tagger-20-header=admin3&tagger-20-tag=%23adm3%2Bname&tagger-21-header=location&tagger-21-tag=%23loc%2Bname&tagger-22-header=latitude&tagger-22-tag=%23geo%2Blat&tagger-23-header=longitude&tagger-23-tag=%23geo%2Blon&tagger-25-header=source&tagger-25-tag=%23meta%2Bsource&tagger-27-header=notes&tagger-27-tag=%23description&tagger-28-header=fatalities&tagger-28-tag=%23affected%2Bkilled&header-row=1')
conflict_loc = conflict_loc.iloc[1:]
conflict_loc.to_csv("data/acled_unprocessed_conflict_locations.csv")'''


'conflict_loc = pd.read_csv(\'https://data.humdata.org/hxlproxy/data.csv?url=https%3A%2F%2Fapi.acleddata.com%2Facled%2Fread.csv%3Flimit%3D0%26iso%3D368&name=ACLEDHXL&tagger-match-all=on&tagger-02-header=iso&tagger-02-tag=%23country%2Bcode&tagger-03-header=event_id_cnty&tagger-03-tag=%23event%2Bcode&tagger-05-header=event_date&tagger-05-tag=%23date%2Boccurred+&tagger-08-header=event_type&tagger-08-tag=%23event%2Btype&tagger-09-header=actor1&tagger-09-tag=%23group%2Bname%2Bfirst&tagger-10-header=assoc_actor_1&tagger-10-tag=%23group%2Bname%2Bfirst%2Bassoc&tagger-12-header=actor2&tagger-12-tag=%23group%2Bname%2Bsecond&tagger-13-header=assoc_actor_2&tagger-13-tag=%23group%2Bname%2Bsecond%2Bassoc&tagger-16-header=region&tagger-16-tag=%23region%2Bname&tagger-17-header=country&tagger-17-tag=%23country%2Bname&tagger-18-header=admin1&tagger-18-tag=%23adm1%2Bname&tagger-19-header=admin2&tagger-19-tag=%23adm2%2Bname&tagger-20-header=admin3&tagger-20-tag=%23adm3%2Bname&tagger-21-header=location&tag

In [215]:
from shapely.geometry import Point
conflict_loc = pd.read_csv('data/acled_unprocessed_conflict_locations.csv')
conflict_loc_geo = gpd.GeoDataFrame(conflict_loc)
points = [Point(float(j), float(i)) for i, j in zip(conflict_loc.latitude, conflict_loc.longitude)]
geometries_acled = gpd.GeoSeries(points).buffer(0.1) # buffer is somewhat arbitrary...
# align geometries - can't know what geometry acled was using...
acled_locations = gpd.GeoDataFrame()
acled_locations['geometry'] = geometries_acled
acled_locations.crs = iq_locations.crs
acled_locations.to_crs(iq_locations.crs)
acled_locations['date'] = conflict_loc['event_date']
acled_locations.to_file('data/acled_conflict_geometries.shp')

In [216]:
acled_temp = gpd.sjoin(iq_locations, acled_locations, 'left')
acled_temp.to_file('data/acled_joined_locations.shp')

# for now randomly assign conflict

In [217]:
#acled_temp.sample(1000).to_file('data/acled_joined_locations_sample.shp')
pop_list  = location_csv['pop_density'] * 100 # some pop
pop_list = [round(i * 10) for i in pop_list]
location_csv['pop/cap'] = [round(i, 0) for i in pop_list]
#location_csv = location_csv.iloc[:,:-1]
import numpy as np
conflict_list = []
conflict_type = []
for i in range(100):
    one = np.random.randint(0,2)
    if one > 0:
        conflict_type.append('conflict')
        one = np.random.randint(1,50)
    else:
        if np.random.randint(0,2) > 0:
            conflict_type.append('camp')
        else:
            conflict_type.append('town')
    conflict_list.append(one)
location_csv['conflict_date'] = conflict_list
location_csv['location_type'] = conflict_type
location_csv_new = location_csv.iloc[:,:-1]
location_csv_new.to_csv('flee-release/testing_locations_data.csv', index = False, header=False)


In [218]:
#locations_sample = gpd.read_file('data/acled_joined_locations_sample.shp')
#conflict_locations = gpd.read_file('data/acled_joined_locations.shp')
#conflict_locations.head()

location_csv_new

,name,region,country,lat,lon,location_type,conflict_date,pop/cap
18147,Al Halfayah,Missan,Iraq,31.820000,47.430000,conflict,25,2523
21307,Jubur Sharrad,Wassit,Iraq,33.030000,45.910000,conflict,19,101
14584,Sharayi,Erbil,Iraq,36.080000,43.580000,camp,0,3334
6066,Kfradoli Lower,Sulaymaniyah,Iraq,36.220000,44.760000,conflict,36,2075
4323,Shawk,Sulaymaniyah,Iraq,35.210000,44.910000,town,0,359
13472,Khela Rasha,Erbil,Iraq,36.730000,45.000000,town,0,155
15439,Malakan Upper,Erbil,Iraq,36.440000,44.610000,conflict,1,243
728,5 Miles,Basrah,Iraq,30.538814,47.764197,camp,0,83842
2139,Hay al-Bado,Muthanna,Iraq,31.534444,45.211389,camp,0,12095
7952,Abo al-'Araeis,Babylon,Iraq,32.506667,44.378889,camp,0,4989


In [22]:
locations_sample = acled_temp.copy()
just_date = []
for i in locations_sample.itertuples():
    try:
        if '-' in i.date:
            just_date.append(int("".join([i for i in i.date if i != '-'])))
        else:
            just_date.append(-1)
    except:
        just_date.append(-1)
            
locations_sample['justdate'] = just_date
locations_sample.sort_values('justdate', inplace = True)
date_rank = []
datessofar = set()
current = -1
for i in locations_sample.itertuples():
    if i.justdate != -1:
        if i.justdate not in datessofar:
            current += 1
            datessofar.add(i.justdate)
        date_rank.append(current)
    else:
        date_rank.append(None)
locations_sample['conflict_start'] = date_rank
    
        
    

In [23]:
conflict_list = []
conflicts = set()
for i in locations_sample.itertuples():
    if i.justdate >= 0:
        if str(i.Longitude) + str(i.Latitude) not in conflicts:
            conflicts.add(str(i.Longitude) + str(i.Latitude))
            conflict_list.append(i)
conflict_df = gpd.GeoDataFrame(conflict_list)
conflict_df

,Index,PCode,PNameEn,PNameAr,Longitude,Latitude,PCodeAlt1,PNameAlt1,A1Code,A1NameEn,A2Code,A2NameEn,geometry,index_right,date,justdate,conflict_start
0,20112,IQ-P20074,Hay al Zahraa',حي الزهراء,43.206363,36.385903,1510962.0,Hay Al Zahraa',IQ-G15,Ninewa,IQ-D087,Mosul,POINT (43.206363 36.385903),6348.0,2017-01-01,20170101,0.0
1,22692,IQ-P22650,Al Bu Dubaigh,,44.472500,31.889722,1709932.0,A`l bu dubaigh,IQ-G17,Najaf,IQ-D099,Manathera,POINT (44.4725 31.889722),6342.0,2017-01-01,20170101,0.0
2,22407,IQ-P22366,Al Saray,,44.403745,32.094010,1710200.0,Al saray,IQ-G17,Najaf,IQ-D098,Kufa,POINT (44.403745 32.09401),6338.0,2017-01-01,20170101,0.0
3,113,IQ-P00114,Al-Zuwayiah,قرية الزوية,43.837899,33.408921,110084.0,Al-Zuwayiah,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.837899 33.408921),6343.0,2017-01-01,20170101,0.0
4,20958,IQ-P20920,Baisan,,43.090000,36.410000,1508668.0,Baisan,IQ-G15,Ninewa,IQ-D091,Tilkaif,POINT (43.09 36.41),6352.0,2017-01-01,20170101,0.0
5,22930,IQ-P22888,Um 'Arief Algharbiya,,44.475278,31.826111,1709879.0,Um `arief algharbiya,IQ-G17,Najaf,IQ-D099,Manathera,POINT (44.475278 31.826111),6342.0,2017-01-01,20170101,0.0
6,23202,IQ-P23161,Sharit al Makrama (Tajawz),,44.355600,32.065200,1710525.0,Sharit Al Makrama (Tajawz),IQ-G17,Najaf,IQ-D100,Najaf,POINT (44.3556 32.0652),6338.0,2017-01-01,20170101,0.0
7,137,IQ-P00138,Hay al Thubbat 4,,43.798484,33.337002,110212.0,Hay Al Thubbat 4,IQ-G01,Anbar,IQ-D002,Falluja,POINT (43.798484 33.337002),6343.0,2017-01-01,20170101,0.0
8,22537,IQ-P22496,Hay 'Adalah 117,,44.356667,32.018611,1710139.0,Hay `adalah 117,IQ-G17,Najaf,IQ-D098,Kufa,POINT (44.356667 32.018611),6338.0,2017-01-01,20170101,0.0
9,22864,IQ-P22822,Hay al-Nasr / 2,,44.488056,31.886944,1709928.0,Hay al-nasr / 2,IQ-G17,Najaf,IQ-D099,Manathera,POINT (44.488056 31.886944),6342.0,2017-01-01,20170101,0.0


In [32]:
'''location_csv_tmp = location_csv.copy()
points = [Point(i.lon, i.lat) for i in location_csv_tmp.itertuples()]
location_csv_tmp['geometry'] = points
conflict_list = []
type_list = []
for i in location_csv_tmp.itertuples():
    for j in conflict_df.itertuples():
        if i.geometry == j.geometry:
            conflict_list.append([i.geometry, j.conflict_start])
            break
    '''

#location_csv_tmp_joined = gpd.sjoin(location_csv_tmp, conflict_df, 'left')
#location_csv_tmp_joined

"location_csv_tmp = location_csv.copy()\npoints = [Point(i.lon, i.lat) for i in location_csv_tmp.itertuples()]\nlocation_csv_tmp['geometry'] = points\nconflict_list = []\ntype_list = []\nfor i in location_csv_tmp.itertuples():\n    for j in conflict_df.itertuples():\n        if i.geometry == j.geometry:\n            conflict_list.append([i.geometry, j.conflict_start])\n            break\n    "